In [ ]:
import os
import torch
import numpy as np
import diffusers
from diffusers import AutoencoderKL

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
test_id = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data/ac_train/mel/-0A1_JR5f34_16000_26000.npy"
test_mel = np.load(test_id)[:,:624]
print(test_mel.shape)
plt.imshow(test_mel)

In [ ]:
vae = AutoencoderKL.from_pretrained("/blob/v-yuancwang/AUDITPLUS/VAEGAN/checkpoint-24000")
vae.requires_grad_(False)

In [ ]:
test_mel_tensor = np.expand_dims(np.expand_dims(test_mel, 0), 0)
test_mel_tensor = torch.Tensor(test_mel_tensor)
test_mel_tensor.shape

In [ ]:
with torch.no_grad():
    posterior = vae.encode(test_mel_tensor).latent_dist
    z = posterior.sample()
    vae_output = vae.decode(z).sample

In [ ]:
vae_res = vae_output[0][0].numpy()
print(vae_res.shape)
plt.imshow(vae_res)

In [ ]:
abs_diff = abs(vae_res - test_mel)
print(abs_diff.max(), abs_diff.min(), abs_diff.mean())

In [ ]:
from IPython.display import Audio
from shutil import copy

In [ ]:
test_wav_id = test_id.replace("/mel/", "/wav/").replace(".npy", ".wav")
Audio(test_wav_id)

In [ ]:
copy(test_id, '/home/v-yuancwang/AUDIT_v2/test_vae_mel/'+'test_0.npy')
np.save('/home/v-yuancwang/AUDIT_v2/test_vae_mel/'+'test_1.npy', vae_res)